In [1]:
import sqlite3
from datetime import datetime

In [2]:
! mkdir -p databases

In [5]:
DB_PATH = "databases/library.db"

In [6]:
connection = sqlite3.connect(DB_PATH)

In [10]:
cursor = connection.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Book(
        id INTEGER PRIMARY KEY,
        author TEXT,
        title TEXT,
        publish_year INTEGER
    );
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Reader(
        id INTEGER PRIMARY KEY,
        name TEXT
    );
''')

cursor.execute('''
        CREATE TABLE IF NOT EXISTS Record(
        id INTEGER PRIMARY KEY,
        reader_id INTEGER,
        book_id INTEGER,
        taking_date TEXT,
        returning_date TEXT,
        FOREIGN KEY (reader_id) REFERENCES Reader (id),
        FOREIGN KEY (book_id) REFERENCES Book (id)
    );
''')

books = [
        ('J.K.Rowling' , 'Harry Potter and the Philosopher''s Stone', 1997),
        ('J.K.Rowling' , 'Harry Potter and the Chamber of Secrets', 1998),
        ('Дж. Оруэлл' , '1984', 1949),
        ('Корявов В.П' , 'Методы решения задач в общем курсе физики (Механика)', 2016),
        ('А.Стругацкий, Б.Стругацкий' , 'Понедельник начинается в субботу', 1964),
]

cursor.executemany(
    """INSERT INTO Book(author, title, publish_year)
       VALUES (?, ?, ?);""",
    books
)

readers = [
    ('I.Ivanov',),
    ('S.Petrov',),
    ('P.Sidorov',),
]

cursor.executemany(
    """INSERT INTO Reader(name)
       VALUES (?);""",
    readers
)


records = [
    (1, 1, '2023-03-01', '2023-04-01'),
    (2, 4, '2023-03-18', '2023-04-18'),
    (3, 3, '2023-03-18', '2023-04-18'),
]

cursor.executemany(
    """INSERT INTO Record(reader_id, book_id, taking_date, returning_date)
       VALUES (?, ?, ?, ?);""",
    records
)
connection.commit()

In [2]:
def select_all(connection, table_name: str) -> None:
    cursor = connection.cursor()

    for row in cursor.execute(f"SELECT * FROM {table_name};"):
        print(row)

In [26]:
select_all(connection, 'Book')
print('\n')
select_all(connection, 'Reader')
print('\n')
select_all(connection, 'Record')

(1, 'J.K.Rowling', 'Harry Potter and the Philosophers Stone', 1997)
(2, 'J.K.Rowling', 'Harry Potter and the Chamber of Secrets by J. K. Rowling', 1998)
(3, 'Дж. Оруэлл', '1984', 1949)
(4, 'Корявов В.П', 'Методы решения задач в общем курсе физики (Механика)', 2016)
(5, 'А.Стругацкий, Б.Стругацкий', 'Понедельник начинается в субботу', 1964)


(1, 'I.Ivanov')
(2, 'S.Petrov')
(3, 'P.Sidorov')


(1, 1, 1, '2023-03-01', '2023-04-01')
(2, 2, 4, '2023-03-18', '2023-04-18')
(3, 3, 3, '2023-03-18', '2023-04-18')


In [30]:
def print_books(connection):
    cursor = connection.cursor()
    for row in cursor.execute("SELECT title FROM Book;"):
        print(row[0])

In [31]:
print_books(connection)

Harry Potter and the Philosophers Stone
Harry Potter and the Chamber of Secrets by J. K. Rowling
1984
Методы решения задач в общем курсе физики (Механика)
Понедельник начинается в субботу


In [32]:
def print_readers(connection):
    cursor = connection.cursor()
    for row in cursor.execute("SELECT name FROM Reader;"):
        print(row[0])

In [33]:
print_readers(connection)

I.Ivanov
S.Petrov
P.Sidorov


In [39]:
def add_book(connection, book: tuple):
    cursor = connection.cursor()
    cursor.executemany(
    """INSERT INTO Book(author, title, publish_year)
       VALUES (?, ?, ?);""",
    book
    )
    connection.commit()

In [46]:
book = [('H.Wells', 'The War of the Worlds', 1897)]
add_book(connection, book)
select_all(connection, 'Book')

(1, 'J.K.Rowling', 'Harry Potter and the Philosophers Stone', 1997)
(2, 'J.K.Rowling', 'Harry Potter and the Chamber of Secrets', 1998)
(3, 'Дж. Оруэлл', '1984', 1949)
(4, 'Корявов В.П', 'Методы решения задач в общем курсе физики (Механика)', 2016)
(5, 'А.Стругацкий, Б.Стругацкий', 'Понедельник начинается в субботу', 1964)
(6, 'H.Wells', 'The War of the Worlds', 1897)


In [47]:
def add_reader(connection, reader:tuple):
    cursor = connection.cursor()
    cursor.executemany(
    """INSERT INTO Reader(name)
       VALUES (?);""",
    reader
    )
    connection.commit()

In [49]:
reader = [('J.Depp',)]
add_reader(connection, reader)
select_all(connection, 'Reader')

(1, 'I.Ivanov')
(2, 'S.Petrov')
(3, 'P.Sidorov')
(4, 'J.Depp')


In [8]:
def give_book(connection, reader: str, book: str, dor: str):
    cursor = connection.cursor()
    
    cursor.execute(f"SELECT * FROM Reader WHERE name = '{reader}'")
    reader_id = cursor.fetchone()[0]
    cursor.execute(f"SELECT * FROM Book WHERE title = '{book}'")
    book_id = cursor.fetchone()[0]
    date = datetime.today().strftime('%Y-%m-%d')
    value = [(reader_id, book_id, date, dor,)]
    cursor.executemany(
    """INSERT INTO Record(reader_id, book_id, taking_date, returning_date)
       VALUES (?, ?, ?, ?);""",
    value
    )
    connection.commit()

In [9]:
give_book(connection, 'I.Ivanov', 'Harry Potter and the Chamber of Secrets', '2023-04-15')

In [15]:
select_all(connection, 'Record')

(1, 1, 1, '2023-03-01', '2023-04-01')
(2, 2, 4, '2023-03-18', '2023-04-18')
(3, 3, 3, '2023-03-18', '2023-04-18')
(4, 1, 2, '2023-04-03', '2023-04-15')


In [16]:
def take_book(connection, reader: str, book: str):
    cursor = connection.cursor()
    cursor.execute(f"SELECT * FROM Reader WHERE name = '{reader}'")
    reader_id = cursor.fetchone()[0]
    cursor.execute(f"SELECT * FROM Book WHERE title = '{book}'")
    book_id = cursor.fetchone()[0]
    cursor.execute(f"DELETE FROM Record WHERE reader_id = '{reader_id}' AND book_id = '{book_id}';")
    connection.commit()

In [17]:
take_book(connection, 'S.Petrov', 'Методы решения задач в общем курсе физики (Механика)')

In [18]:
select_all(connection, 'Record')

(1, 1, 1, '2023-03-01', '2023-04-01')
(3, 3, 3, '2023-03-18', '2023-04-18')
(4, 1, 2, '2023-04-03', '2023-04-15')
